# Web Scraping: Titulares
Según el procedimiento en https://www.datacamp.com/community/tutorials/web-scraping-using-python

In [1]:
#Importes necesarios
import pymysql as sql
import pymysql.cursors
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime

In [2]:
#Conectar a la base de datos en MySQL local
conn = sql.connect(
    host="localhost",
    port=3306,
    user="root",
    passwd="",
    db="siglonuevo")
cursor = conn.cursor()
cursorclass = pymysql.cursors.DictCursor

In [10]:
#Establecer función para el cambio de fechas
def fecha_variable(start_date):
    fecha = start_date.strftime("%Y-%m-%d")
    url = "https://www.elsiglodetorreon.com.mx/archivo/?d=" + fecha
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    #for nota in soup.findAll('div', {'class' : 'col-xs-9'}):
        #titular = nota.find('a')
        #.get_text().strip()
        #href = titular['href']
        #print(href, titular.get_text())

    with conn.cursor() as cursor:
        for nota in soup.findAll('div', {'class' : 'col-xs-9'}):
            titular = nota.find('a')
            titular_texto = titular.get_text().strip()
            enlace = titular['href']
            enlace_completo = "https://www.elsiglodetorreon.com.mx" + enlace
            #.get_text().strip()
            orden = "INSERT INTO `titulares` (`titulares`, `fecha`, `enlace` ) VALUES (%s, %s, %s)"
            cursor.execute(orden, (titular_texto, fecha, enlace_completo))
    conn.commit()

In [11]:
#Variabilización de las fechas y ejecución de la función

start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 4, 16)
delta = datetime.timedelta(days=1)

while start_date <= end_date:
    fecha_variable(start_date)
    start_date += delta